In [ ]:
#| echo: false

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import os
import sys
from models import ParityMLP, ParityLSTM
from data import ParityDataset, possible_combinations
from trainer import Trainer
from visualization import plot_history

from data_generator import BooleanDatasetGenerator

%load_ext autoreload
%autoreload 2

Boolean functions, which map binary inputs to binary outputs, are fundamental to digital computing. Understanding how neural networks learn and represent these functions is essential for advancing machine learning applications. This project focuses on training neural networks to accurately learn multiple Boolean functions. Specifically, we aim to determine how closely different neural network architectures can approximate the ideal representations of these functions. Our objective is to analyze the degree of this approximation and its impact on the networks' ability to generalize beyond the training data. By addressing these questions, we seek to enhance our understanding of the capabilities and limitations of neural networks in representing Boolean functions.

### Boolean Functions

Boolean functions are mathematical functions that take binary inputs and produce binary outputs. They are commonly used in computer science and digital electronics to model logical operations. Boolean functions are defined by their truth tables, which specify the output for each possible combination of inputs, @tbl-truthtable shows the truth table for the AND, OR, and XOR functions with two binary inputs. 

<div style="width:80%; margin:auto">

::: {#tbl-truthtable layout-ncol=3}
| X | Y | output |
|------|------|:------:|
| 0    | 0    | 0    |
| 0    | 1    | 0    |
| 1    | 0    | 0    |
| 1    | 1    | 1    |

: AND {#tbl-first}

| X | Y | Output |
|------|------|:------:|
| 0    | 0    | 0    |
| 0    | 1    | 1    |
| 1    | 0    | 1    |
| 1    | 1    | 1    |

: OR {#tbl-second}

| X | Y | Output |
|------|------|:------:|
| 0    | 0    | 0    |
| 0    | 1    | 1    |
| 1    | 0    | 1    |
| 1    | 1    | 0    |

: XOR {#tbl-third}

Truth table for AND, OR, and XOR boolean functions.
:::

</div>

### Dataset

When working with an input size of $N$ bits, the AND and OR functions can require a change of up to $N$ bits to switch the output from 0 to 1 or vice versa. However, the XOR function only needs a change in one bit to achieve the same output switch. This property makes XOR a more complex learning function than AND and OR. Due to this, we will emphasize XORs in this article. Our dataset comprises XOR functions with 16, 32, and 64-bit input sizes.

<div style="width:60%; margin:auto">

| Bits | Size |
|------|------|
| 16   | 65,536 |
| 32   | 4,294,967,296 |
| 64   | 18,446,744,073,709,551,616 |

: Number of combinations for different input sizes {#tbl-combinations}

</div>

For the different length of bits, the total input size is shown in @tbl-combinations. The dataset is created by randomly sampling input combinations and calculating the corresponding outputs using the XOR function. It's worth noting that an input size of $N$ bits results in $2^N$ possible input combinations.

### Perceptron

Perceptron is the simplest form of a neural network. It consists of a single layer of input and output nodes. The perceptron's output is calculated by taking a weighted sum of the input values and passing it through an activation function. The weights are adjusted during training to minimize errors between the predicted and actual outputs. 

![Perceptron](./assets/images/perceptron.png){#fig-perceptron fig-align="center" width=60%}


::: {.callout-note}

The perceptron can learn linearly separable functions, such as the AND and OR functions, but it cannot learn non-linear functions like the XOR function.

:::

For the small input of boolean functions AND and OR, we can adjust the weights and bias to learn the function as shown in @fig-perceptrongates.

![AND, OR, and NOT gates represented using a perceptron](./assets/images/perceptron-gates.jpeg){#fig-perceptrongates fig-align="center" width=60%}


The number of weights and biases increases exponentially for boolean functions with larger input sizes. This makes it difficult to learn complex tasks with a perceptron, and we need more complicated neural network architectures to understand these functions. In this project, we explore Multi-layer perceptron, Recurrent Neural Networks, and Kolmogorov-Arnold Networks.

### Multi-Layer Perceptron

A multi-layer perceptron (MLP) is a neural network with multiple layers of nodes, including an input layer, one or more hidden layers, and an output layer. Each node in the different layers is a perceptron using different activation functions. The hidden layers allow the network to learn non-linear functions by introducing non-linear activation functions, such as the sigmoid or ReLU function. The network weights and biases are adjusted during training using backpropagation to minimize the error between the predicted and actual outputs. 

In a multi-layer perceptron that contains only one hidden layer, the minimum number of neurons required to learn the XOR function of $N$ input size is $N^{1/2}$ [see @irprsm1997]. Moreover, the best-known implementation has $O(N)$ neurons. Considering this, we train two MLPs with different hidden layer sizes to learn XOR functions. The first MLP has only one hidden layer with $4N$ neurons. 

![MLP with one hidden layer.](./assets/images/shallow-mlp.png){#fig-shallow-mlp fig-align="center" width=60%}


@fig-deep-mlp shows an example of the second MLP, which has four hidden layers, each with $N$ number of neurons.

![MLP with four hidden layer.](./assets/images/deep-mlp.png){#fig-deep-mlp fig-align="center" width=60%}

The total number of parameters in the two networks is close, and our goal is to compare the performance of these networks in learning XOR functions over different input size and observe the impact of the hidden layers.

We train both MLPs using the Adam optimizer with a learning rate of 0.001 and a batch size of 32. The networks are trained for 50 epochs, and the loss is calculated using the binary cross-entropy loss function. We randomly sample 10,000, 20,000, and 40,000 training input combinations for the 16, 32, and 64-bit XOR functions, respectively. We evaluate the networks on a separate test set of 50,000, 200,000, and 400,000 input combinations for the 16, 32, and 64-bit XOR functions, respectively.

![MLP with 1 hidden layer performance](./assets/images/shallow-mlp-performance.png){#fig-shallow-mlp-performance fig-align="center" width=80%}

The performance of the MLP with one hidden layer is shown in @fig-shallow-mlp-performance. The network loss and accuracy are going up and down during training, indicating that the network is not learning the XOR function effectively. Moreover, the accuracy over the test set is close to 0.5, which is the accuracy of a random guess. 

![MLP with 4 hidden layers performance](./assets/images/deep-mlp-performance.png){#fig-shallow-mlp-performance fig-align="center" width=80%}

In the case of the MLP with four hidden layers, the network loss performance for input size of 32 and 64 bits is somewhat similar to the model with one hidden layer. However, the model performance over the 16 bits input size is significantly better than the one layer MLP. Moreover, the accuracy for 16 bit input over the test set is 0.97.

### Recurrent Neural Networks

Recurrent Neural Networks (RNNs) are a type of neural network that can process data sequences. They accept the output of the previous step as input to the current step along with the current input, which allows RNNs to maintain a state or memory of the prior steps, making them suitable for processing sequential data. In this project, we formulate the XOR function as a sequence modeling problem, processing the input in small groups of bits at a time. 

Parity functions(XORs) return a value of one if the number of ones in the input value is odd and zero if the number of ones is even. This unique characteristic of XORs allows us to view the inputs as a sequence of small groups. In this experiment, we delve into the problem-solving aspect of how to represent the boolean using recurrent neural networks by processing small input units.

![RNNs for boolean functions](./assets/images/rnn.gif){#fig-rnn fig-align="center" width=80%}

Processing input of length N as a group of small units simplifies the complexity analysis over the network. Considering each input element as a separate sequence unit effectively converts the $N$-bit problem into a 2-bit problem, one coming from the input and the other from the layer's output. Furthermore, we set the initial state of the RNN to zero, as shown in @fig-rnn.

For different input sizes, we train the RNN with 1000 training set, 10000 validation set. Moreover, the testing set contains 50000, 200000, and 400000 for 16, 32, and 64 bits, respectively. The models are trained using the Adam optimizer with a learning rate of 0.01 and a batch size of 8. The model's performance is evaluated using the binary cross-entropy loss and accuracy metrics.

The RNN model did not converge to the ideal XOR function after training over different input sizes of 16, 32, and 64 bits. The model's performance was poor, failing to learn the XOR function with fluctuating loss and accuracy, as shown in @fig-rnn-performance. 

![RNNs model performance](./assets/images/rnn-performance.png){#fig-rnn-performance fig-align="center" width=80%}

All the input values are 0 and 1, as a result diminishing gradients may be causing the model to struggle to learn the XOR function. To address this issue, we employ gated RNNs, such as Long Short-Term Memory (LSTM) and Gated Recurrent Unit (GRU), designed to mitigate the vanishing gradient problem.

![LSTM model performance](./assets/images/lstm-performance.png){#fig-lstm-performance fig-align="center" width=80%}

The LSTM model outperformed the RNN model, achieving a significantly lower loss and higher accuracy over 16, 32, and 64-bit input sizes, as shown in @fig-lstm-performance. The model achieves a perfect representation of the randomly sampled dataset, indicating that the LSTM model can learn the XOR function more effectively with larger input sizes.

::: {.callout-note}

Perfect representation is when a model learns to predict the output of a boolean function with 100% accuracy.

:::

### Kolmogorov-Arnold Networks

Kolmogorov–Arnold Networks (KANs) are a novel neural network architecture inspired by the Kolmogorov-Arnold representation theorem[see @liu2024kan]. Unlike Multilayer Perceptrons (MLPs), which have fixed node activation functions, KANs have learnable activation functions on edges. As a result, KANs have no linear weight matrices; a learnable 1D function replaces each weight parameter parameterized as a spline. @fig-mlp-v-kan shows the difference between MLP and KAN architectures in detail.

![MLP and KANs](./assets/images/mlp-v-kan.png){#fig-mlp-v-kan fig-align="center" width=80%}

Due to their unique architecture, KANs can be powerful tools for learning Boolean functions. In this section, we experiment with KANs and analyze their behavior in contrast with MLP in the context of boolean functions.

We start by implementing a simple KAN that accepts 8 bit inputs and outputs either 0 or 1 according to the XOR function. @fig-kan8 shows the model before training. There are 256 possible input combinations for 8 bit input, and we use 0.3 test set ratio for evaluation.

![8-bit input KANs](./assets/images/kan-8bit.png){#fig-kan8 fig-align="center" width=80%}

After training the activation functions of the KAN are updated as shown in @fig-trained-kan8. Moreover, the models can predict with 61% accuracy over the test set.

![Trained 8-bit input KANs](./assets/images/trained-kan-8bit.png){#fig-trained-kan8 fig-align="center" width=80%}

We further extend the KAN further to 16 bit input size and train the model with 0.3 test set ratio over 30,000 size dataset, the model achieves 53% accuracy. This is not close to a perfect representation, however doing extensive experiments and analysis with different hyperparameter and architectures is part of our future work.


### Results & Summary

In this project, we explored the capabilities of different neural network architectures in learning XOR functions of varying input sizes. We trained Multi-layer perceptrons, Recurrent Neural Networks, and Kolmogorov-Arnold Networks to learn XOR functions with 16, 32, and 64-bit input sizes. Architectures that process the input as a smaller group of bits, such as LSTM, are more effective in learning XOR functions. The number of hidden layers in MLPs can also impact the network's performance.

<div style="width:85%; margin:auto">

| Model | Number of bits | Training Size | Test Size | Accuracy |
|------|------|------|------|------|
| MLP (1 hidden layer) | 16 | 10,000 | 50,000 | 0.50 |
| MLP (1 hidden layer) | 32 | 20,000 | 200,000 | 0.50 |
| MLP (1 hidden layer) | 64 | 40,000 | 400,000 | 0.50 |
| MLP (4 hidden layers) | 16 | 10,000 | 50,000 | 0.97 |
| MLP (4 hidden layers) | 32 | 20,000 | 200,000 | 0.50 |
| MLP (4 hidden layers) | 64 | 40,000 | 400,000 | 0.50 |
| RNN | 16 | 1,000 | 50,000 | 0.49 |
| RNN | 32 | 1,000 | 200,000 | 0.48 |
| RNN | 64 | 1,000 | 400,000 | 0.52 |
| LSTM | 16 | 1,000 | 50,000 | 1.00 |
| LSTM | 32 | 1,000 | 200,000 | 1.00 |
| LSTM | 64 | 1,000 | 400,000 | 1.00 |
| KAN  | 8 | 179 | 77 | 0.61 | 
| KAN  | 16 | 20,000 | 10,000 | 0.53 |

: Performance summary of different neural network architectures in learning XOR functions. {#tbl-results}

</div>

The result summary in @tbl-results shows that the LSTM model outperformed the other model in learning XOR functions, achieving a perfect representation of the dataset. The MLP with four hidden layers performed better than the MLP with one hidden layer, indicating that the number of hidden layers can impact the network's performance. The KANs didn't show promising results in learning XOR functions, however, further analysis is required to understand the behavior of KANs and more specifically their performance in learning boolean functions.



### References

::: {#refs}
:::